In [ ]:
!pip install scipy
!pip install matplotlib
!pip install pydmd
!pip install pykoopman
!pip install imageio
!pip install os
!pip install optht
!pip install derivative
!pip install lightning
!pip install imread
!pip install future
!pip install sphinx


import os
import matplotlib.pyplot as plt
import warnings
import scipy
import imageio
warnings.filterwarnings('ignore')

from pydmd import DMD
from matplotlib import animation
from IPython.display import HTML
import numpy as np
from os import listdir
import pykoopman as pk
import pandas as pd
from imread import imread

%matplotlib inline

import future
import sphinx

from sklearn.decomposition import PCA


from sklearn.preprocessing import StandardScaler
from pydmd import DMDc
from numpy.testing import assert_array_almost_equal

from numpy import linalg as LA
from pydmd import CDMD

from sklearn.datasets import fetch_openml
import sys
import cv2 as cv
import plotly.io as pio
import plotly.graph_objs as go
from PIL import Image
from skimage import color
from plotly import subplots
from sklearn.model_selection import train_test_split
pio.renderers.default = "colab"
from sklearn.decomposition import SparsePCA
import seaborn as sns

In [ ]:
IMG_DIR = '/content/clock'
X = []
X_flat = []
count = 1
size1 = 320
size2 = 148
total = 140
print("Loading...")

for img in sorted(os.listdir(IMG_DIR)):
    if count == total + 1:
        break
    sys.stdout.write("\r" + str(count) + " / " + str(total))
    sys.stdout.flush()
    img_array = cv.imread(os.path.join(IMG_DIR, img), cv.IMREAD_GRAYSCALE,)
    img_pil = Image.fromarray(img_array)
    img_156x120 = np.array(img_pil.resize((size1, size2), Image.ANTIALIAS))
    X.append(img_156x120)
    img_array = img_156x120.flatten()
    X_flat.append(img_array)
    count += 1
print()
print("Done!")

In [ ]:
X_flat = np.asarray(X_flat)
X_flat.shape

In [ ]:
plt.figure(figsize=(10,5));
#plt.imshow(X_flat[89].reshape(148, 320), cmap = plt.cm.gray, interpolation='nearest',
#              clim=(0, 255));
#plt.axis('off');
#plt.tight_layout()

plt.figure(figsize=(8,4));
plt.imshow(X_flat[89].reshape(148, 320), cmap = plt.cm.gray, interpolation='nearest', clim=(0, 255));

In [ ]:
#pca = PCA(0.95)

In [ ]:
PC = pca.fit_transform(X_flat)
eigenvalues = pca.explained_variance_
explained_variance_ratio = pca.explained_variance_ratio_

In [ ]:
PC.shape

In [ ]:
num_components = len(explained_variance_ratio)
plt.bar(range(1, num_components + 1), explained_variance_ratio)
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.show()

# Summary table
cumulative_explained_variance_ratio = np.cumsum(explained_variance_ratio)
summary_table = np.column_stack((range(1, num_components + 1), explained_variance_ratio, cumulative_explained_variance_ratio))
print('PC\tExplained Variance Ratio\tCumulative Explained Variance Ratio')
for row in summary_table:
    print('\t'.join(str(x) for x in row))

In [ ]:
PC1 = PC[0:104]
PC1.shape

In [ ]:
Z = PC.T

# Perform DMD
dmd = DMD(svd_rank=52)
dmd.fit(Z)

# Access DMD modes and dynamics
modes = dmd.modes.T
dynamics = dmd.dynamics

# Visualize DMD modes and dynamics
plt.figure(figsize=(12, 4))

# Plot DMD modes
plt.subplot(1, 2, 1)
for mode in modes:
    plt.plot(mode)
plt.title('DMD Modes')

# Plot DMD dynamics
plt.subplot(1, 2, 2)
for dynamic in dynamics:
    plt.plot(dynamic)
plt.title('DMD Dynamics')

plt.tight_layout()
plt.show()

In [ ]:
dmd_x = DMD(svd_rank=140)
dmd_x.fit(X_flat.T)

for dynamic in dmd_x.dynamics:
    plt.plot(dynamic)
plt.xlim(0, 60)
plt.ylim(-4000, 4000)
plt.title('DMD Dynamics')

In [ ]:
def explainedVariance(percentage, images):
    # percentage should be a decimal from 0 to 1
    pca = PCA(percentage)
    pca.fit(images)
    components = pca.transform(images)
    dmd = DMD(svd_rank=140)
    dmd.fit(components.T)
    approxOriginal = pca.inverse_transform(dmd.reconstructed_data.T)
    approxOriginal = approxOriginal.T.astype(np.float)
    return approxOriginal

In [ ]:
x_recon = dmd.reconstructed_data.T
x_recon = pca.inverse_transform(x_recon)
x_recon = x_recon.astype(np.float)

In [ ]:
approximate = dmd.reconstructed_data.T
approximate = approximate.astype(np.float)

In [ ]:
x_recon.shape

In [ ]:
plt.figure(figsize=(8,4));

# Original Image
plt.subplot(1, 2, 1);
plt.imshow(X_flat[3].reshape(148,320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('', fontsize = 14)
plt.title('Original Image', fontsize = 20);

# 154 principal components
plt.subplot(1, 2, 2);
plt.imshow(x_recon.T[3].reshape(148, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('', fontsize = 14)
plt.title('DMD Reconstruction', fontsize = 20);

In [ ]:
plt.figure(figsize=(20,4));

plt.subplot(1, 5, 1);
plt.imshow(explainedVariance(.80, X_flat)[:,1].reshape(148, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('15 Components', fontsize = 12)
plt.title('80% of Explained Variance', fontsize = 14);

# 154 principal components
plt.subplot(1, 5, 2);
plt.imshow(explainedVariance(.9, X_flat)[:,1].reshape(148, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('34 Components', fontsize = 12)
plt.title('90% of Explained Variance', fontsize = 14);

plt.subplot(1, 5, 3);
plt.imshow(explainedVariance(.95, X_flat)[:,1].reshape(148, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('52 Components', fontsize = 12)
plt.title('95% of Explained Variance', fontsize = 14);

plt.subplot(1, 5, 4);
plt.imshow(explainedVariance(.99, X_flat)[:,1].reshape(148, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('104 Components', fontsize = 12)
plt.title('99% of Explained Variance', fontsize = 14);

# Original Image
plt.subplot(1, 5, 5);
plt.imshow(X_flat[1].reshape(148,320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.xlabel('', fontsize = 14)
plt.title('Original Image', fontsize = 20);

In [ ]:
i = 0
total = 140

for i in range(0, 140, 1):
    if i == total + 1:
       break
    images_dir = '/content/ReconstructionClock95'
    plt.figure(figsize=(8,4));
    plt.imshow(x_recon[i].reshape(148, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
    plt.savefig(f"{images_dir}/label{[i]}.png")
    #plt.savefig('approximate[i].png')

In [ ]:
import shutil
shutil.make_archive('/content/ReconstructionClock95', 'zip', '/content/ReconstructionClock95')

In [ ]:
plt.figure(figsize=(10,5));
plt.imshow(x_recon[89].reshape(148, 320), cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
plt.axis('off');
plt.tight_layout()